# Biomaker CA: Evolving in small worlds

This colab shows the experiments performed in small worlds. See the [related YouTube video here](https://youtu.be/QaCH4ATmt_s?si=3osYC9WuAL6J89qP).

I used the 'persistence' configuration, but feel free to experiment with different environments.

I haven't performed a serious ablation study, but I nevertheless managed to evolve a biome that seems to be stable in such small worlds (h=16).

Copyright 2023 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

## Imports

In [ ]:
#@title install selforg package
# install the package locally
!pip install --upgrade -e git+https://github.com/google-research/self-organising-systems.git#egg=self_organising_systems&subdirectory=biomakerca
# activate the locally installed package (otherwise a runtime restart is required)
import pkg_resources
import importlib
# Reload the resources because we uninstalled and reinstalled some packages.
importlib.reload(pkg_resources)
pkg_resources.get_distribution("self_organising_systems").activate()

In [ ]:
#@title imports & notebook utilities
from self_organising_systems.biomakerca import environments as evm
from self_organising_systems.biomakerca.agent_logic import BasicAgentLogic
from self_organising_systems.biomakerca.mutators import BasicMutator
from self_organising_systems.biomakerca.mutators import RandomlyAdaptiveMutator
from self_organising_systems.biomakerca.step_maker import step_env
from self_organising_systems.biomakerca.display_utils import zoom, tile2d
from self_organising_systems.biomakerca.custom_ipython_display import display
from self_organising_systems.biomakerca.env_logic import process_structural_integrity_n_times
from self_organising_systems.biomakerca.utils import save_dna, load_dna

import cv2
import numpy as np
import jax.random as jr
import jax.numpy as jp
from jax import vmap
from jax import jit
import jax
import time

import tqdm
import mediapy as media
from functools import partial
from IPython.display import clear_output
import matplotlib.pyplot as plt


def pad_text(img, text):
  font = cv2.FONT_HERSHEY_SIMPLEX
  orgin = (5, 15)
  fontScale = 0.5
  color = (0, 0, 0)
  thickness = 1

  # ensure to preserve even size (assumes the input size was even.
  new_h = img.shape[0]//15
  new_h = new_h if new_h % 2 == 0  else new_h + 1
  img = np.concatenate([np.ones([new_h, img.shape[1], img.shape[2]]), img], 0)
  img = cv2.putText(img, text, orgin, font, fontScale, color, thickness, cv2.LINE_AA)
  return img


def visualize_run(key, programs, agent_logic, mutator, st_env, config, n_steps,
                  init_steps_per_frame=2,
                  when_to_double_speed=[100, 500, 1000, 2000, 5000],
                  fps=20,
                  zoom_sz=40):
  env = st_env
  steps_per_frame = init_steps_per_frame

  def make_frame(env):
    return zoom(evm.grab_image_from_env(env, config), zoom_sz)

  frame = make_frame(env)

  out_file = "video.mp4"
  with media.VideoWriter(out_file, shape=frame.shape[:2], fps=fps,
                         crf=18) as video:
    for i in tqdm.trange(n_steps):
      if i in when_to_double_speed:
        steps_per_frame *= 2

      key, ku = jr.split(key)
      env, programs = step_env(
          ku, env, config, agent_logic, programs, do_reproduction=True,
            mutate_programs=True, mutator=mutator)

      if i % steps_per_frame == 0:
        video.add_image(make_frame(env))

  media.show_video(media.read_video(out_file))


## Select the configuration, the agent logic and the mutator

In [ ]:
ec_id = "persistence" #@param ['persistence', 'pestilence', 'collaboration', 'sideways']
env_width_type = "landscape" #@param ['wide', 'landscape', 'square', 'petri']
# original h is 72
h = 16 #@param {type: "integer"}

env_and_config = evm.get_env_and_config(ec_id, width_type=env_width_type, h=h)
st_env, config = env_and_config
w = st_env.type_grid.shape[1]

agent_model = "extended" #@param ['minimal', 'extended']
agent_logic = BasicAgentLogic(config, minimal_net=agent_model=="minimal")

mutator_type = "randomly_adaptive" #@param ['basic', 'randomly_adaptive']
sd = 1e-2 if mutator_type == "basic" and agent_model == "basic" else 1e-3
mutator = (BasicMutator(sd=sd, change_perc=0.2) if mutator_type == "basic"
           else RandomlyAdaptiveMutator(init_sd=sd, change_perc=0.2))

## Example initial run
This is expected to fail.

In [ ]:
key = jr.PRNGKey(43)


# How many unique programs (organisms) are allowed in the simulation.
N_MAX_PROGRAMS = 128

n_steps = 5000

ku, key = jr.split(key)
programs = vmap(agent_logic.initialize)(jr.split(ku, N_MAX_PROGRAMS))
ku, key = jr.split(key)
programs = vmap(mutator.initialize)(jr.split(ku, programs.shape[0]), programs)

ku, key = jr.split(key)
visualize_run(ku, programs, agent_logic, mutator, st_env, config, n_steps)



## Try all possible logics/mutators
This is also expected to fail.

In [ ]:

agent_logics = [
    BasicAgentLogic(config, minimal_net=True),
    BasicAgentLogic(config, minimal_net=False),
    BasicAgentLogic(config, minimal_net=True),
    BasicAgentLogic(config, minimal_net=False)
]

mutators = [
    BasicMutator(sd=1e-2, change_perc=0.2),
    BasicMutator(sd=1e-3, change_perc=0.2),
    RandomlyAdaptiveMutator(init_sd=1e-3, change_perc=0.2),
    RandomlyAdaptiveMutator(init_sd=1e-3, change_perc=0.2),
]


n_steps = 5000
N_MAX_PROGRAMS = 128

steps_per_frame = 2
# on what STEP to double speed.
# NOTE that it is different from previous colabs, where we were talking about
# frames.
when_to_double_speed = [100, 500, 1000, 2000, 5000]
fps = 20
# this affects the size of the image. If this number is not even, the resulting
# video *may* not be supported by all renderers.
zoom_sz = 20

b_programs = []
for i in range(len(agent_logics)):
  ku, key = jr.split(key)
  programs = vmap(agent_logics[i].initialize)(jr.split(ku, N_MAX_PROGRAMS))
  ku, key = jr.split(key)
  programs = vmap(mutators[i].initialize)(jr.split(ku, programs.shape[0]), programs)
  b_programs.append(programs)

envs = [st_env] * len(agent_logics)


def make_frame(envs):
  return zoom(
      tile2d([
          np.pad(evm.grab_image_from_env(env, config), ((1,0), (1,0), (0,0)), constant_values=1.)
          for env in envs], w=2)[1:,1:],
      zoom_sz)

step = 0

frame = make_frame(envs)
out_file = "video.mp4"


with media.VideoWriter(out_file, shape=frame.shape[:2], fps=fps, crf=18) as video:
  video.add_image(frame)
  for i in tqdm.trange(n_steps):
    if i in when_to_double_speed:
      steps_per_frame *= 2
    new_envs, new_b_programs = [] , []
    for k in range(len(agent_logics)):

      key, ku = jr.split(key)
      env, programs = step_env(
          ku, envs[k], config, agent_logics[k], b_programs[k], do_reproduction=True,
            mutate_programs=True, mutator=mutators[k])
      new_envs.append(env)
      new_b_programs.append(programs)
    envs = new_envs
    b_programs = new_b_programs

    if i % steps_per_frame == 0:
      video.add_image(make_frame(envs))

media.show_video(media.read_video(out_file))

## Repeat one model several times

Note: we could parallelize the computation here with vmaps. I don't to make sure that we won't run out of memory, in general. But in this specific case, we probably would be fine.

In [ ]:
agent_model = "extended" #@param ['minimal', 'extended']
agent_logic = BasicAgentLogic(config, minimal_net=agent_model=="minimal")

mutator_type = "randomly_adaptive" #@param ['basic', 'randomly_adaptive']
sd = 1e-2 if mutator_type == "basic" and agent_model == "basic" else 1e-3
mutator = (BasicMutator(sd=sd, change_perc=0.2) if mutator_type == "basic"
           else RandomlyAdaptiveMutator(init_sd=sd, change_perc=0.2))

In [ ]:


n_steps = 10000
N_MAX_PROGRAMS = 128
N_REPS = 9

steps_per_frame = 2
# on what STEP to double speed.
# NOTE that it is different from previous colabs, where we were talking about
# frames.
when_to_double_speed = [100, 500, 1000, 2000, 5000]
fps = 20
# this affects the size of the image. If this number is not even, the resulting
# video *may* not be supported by all renderers.
zoom_sz = 20

b_programs = []
for i in range(N_REPS):
  ku, key = jr.split(key)
  programs = vmap(agent_logic.initialize)(jr.split(ku, N_MAX_PROGRAMS))
  ku, key = jr.split(key)
  programs = vmap(mutator.initialize)(jr.split(ku, programs.shape[0]), programs)
  b_programs.append(programs)

envs = [st_env] * N_REPS

def make_frame(envs):
  return zoom(
      tile2d([
          np.pad(evm.grab_image_from_env(env, config), ((1,0), (1,0), (0,0)), constant_values=1.)
          for env in envs])[1:,1:],
      zoom_sz)

step = 0

frame = make_frame(envs)
out_file = "video.mp4"


with media.VideoWriter(out_file, shape=frame.shape[:2], fps=fps, crf=18) as video:
  video.add_image(frame)
  for i in tqdm.trange(n_steps):
    if i in when_to_double_speed:
      steps_per_frame *= 2
    new_envs, new_b_programs = [] , []
    for k in range(N_REPS):

      key, ku = jr.split(key)
      env, programs = step_env(
          ku, envs[k], config, agent_logic, b_programs[k], do_reproduction=True,
            mutate_programs=True, mutator=mutator)
      new_envs.append(env)
      new_b_programs.append(programs)
    envs = new_envs
    b_programs = new_b_programs

    if i % steps_per_frame == 0:
      video.add_image(make_frame(envs))

media.show_video(media.read_video(out_file))

## End-to-end meta-evolution: 5000 steps

In [ ]:
agent_model = "extended" #@param ['minimal', 'extended']
agent_logic = BasicAgentLogic(config, minimal_net=agent_model=="minimal")

mutator_type = "randomly_adaptive" #@param ['basic', 'randomly_adaptive']
sd = 1e-2 if mutator_type == "basic" and agent_model == "basic" else 1e-3
mutator = (BasicMutator(sd=sd, change_perc=0.2) if mutator_type == "basic"
           else RandomlyAdaptiveMutator(init_sd=sd, change_perc=0.2))

In [ ]:
def count_agents_f(env, etd):
  return etd.is_agent_fn(env.type_grid).sum()

@partial(jit, static_argnames=["config", "agent_logic", "mutator", "n_steps", "n_max_programs"])
def evaluate_biome(key, st_env, config, agent_logic, mutator, n_steps,
                   init_program=None, n_max_programs=128):
  def body_f(i, carry):
    key, env, programs, tot_agents_n = carry
    ku, key = jr.split(key)

    env, programs = step_env(
        ku, env, config, agent_logic, programs, do_reproduction=True,
          mutate_programs=True, mutator=mutator)

    tot_agents_n += count_agents_f(env, config.etd)
    return key, env, programs, tot_agents_n

  if init_program is None:
    ku, key = jr.split(key)
    programs = vmap(agent_logic.initialize)(jr.split(ku, n_max_programs))
    ku, key = jr.split(key)
    programs = vmap(mutator.initialize)(jr.split(ku, programs.shape[0]), programs)
  else:
    programs = jp.repeat(init_program[None, :], n_max_programs, axis=0)

  key, env, programs, tot_agents_n = jax.lax.fori_loop(
      0, n_steps, body_f, (key, st_env, programs, 0))

  # check whether they got extinct:
  is_extinct = (count_agents_f(env, config.etd) == 0).astype(jp.int32)
  return tot_agents_n, is_extinct

In [ ]:
from evojax.algo import PGPE

key = jr.PRNGKey(137)

N_MAX_PROGRAMS = 128

n_steps = 5000

# initialize params
ku, key = jr.split(key)
init_program = agent_logic.initialize(ku)
ku, key = jr.split(key)
init_program = mutator.initialize(ku, init_program)


pop_size = 32
ku, key = jr.split(key)
solver = PGPE(
    pop_size=pop_size,
    param_size=init_program.shape[0],
    optimizer='adam',
    center_learning_rate=0.001,
    stdev_learning_rate=0.001,
    stdev_max_change=0.002,
    seed=ku[0],
    init_params=init_program,
    init_stdev=0.001,
)

n_max_programs = 64 # less than usual, but it doesn't really matter.
death_penalty = 1e6 * n_steps // 1000

@jit
def v_fitness_f(key, v_params):
  ku, key = jr.split(key)
  tot_agents_n, num_deaths = vmap(partial(
      evaluate_biome, st_env=st_env, config=config, agent_logic=agent_logic,
      mutator=mutator, n_steps=n_steps, n_max_programs=n_max_programs))(
          jr.split(key, pop_size), init_program=v_params)
  fitness = tot_agents_n - num_deaths * death_penalty
  return fitness

mean_fit_log = []
max_fit_log = []

In [ ]:

for _ in range(30):
  # sample
  programs = solver.ask()
  # eval
  key, k1 = jr.split(key)
  fitness = v_fitness_f(k1, programs)
  # update
  solver.tell(fitness)

  mean_fitness = fitness.mean()
  max_fitness = fitness.max()
  mean_fit_log.append(mean_fitness)
  max_fit_log.append(max_fitness)
  print(mean_fitness, max_fitness)
  if len(mean_fit_log) % 10 == 0:
    clear_output()
    plt.plot(mean_fit_log, label="mean_fitness")
    plt.plot(max_fit_log, label="max_fitness")
    plt.legend()
    plt.show()

### Show an example run of the result

Consider modifying the code to vary the extent of the simulation and video configs.

In [ ]:

key = jr.PRNGKey(43)

# How many unique programs (organisms) are allowed in the simulation.
N_MAX_PROGRAMS = 128

n_steps = 20000

programs = jp.repeat(solver.best_params[None,:], N_MAX_PROGRAMS, axis=0)


env = st_env

steps_per_frame = 2
# on what STEP to double speed.
# NOTE that it is different from previous colabs, where we were talking about
# frames.
when_to_double_speed = [100, 500, 1000, 2000, 5000]

stop_at = [5000]

fps = 20
# this affects the size of the image. If this number is not even, the resulting
# video *may* not be supported by all renderers.
zoom_sz = 40

def make_frame(env):
  return zoom(evm.grab_image_from_env(env, config), zoom_sz)

frame = make_frame(env)

out_file = "video.mp4"
with media.VideoWriter(out_file, shape=frame.shape[:2], fps=fps,
                       crf=18,
                       ) as video:
  for i in tqdm.trange(n_steps):
    if i in when_to_double_speed:
      steps_per_frame *= 2

    key, ku = jr.split(key)
    env, programs = step_env(
        ku, env, config, agent_logic, programs, do_reproduction=True,
          mutate_programs=True, mutator=mutator)

    if i % steps_per_frame == 0:
      video.add_image(make_frame(env))
    if i in stop_at:
      # add some extra frames.
      frame = make_frame(env)
      for _ in range(20):
        video.add_image(frame)

media.show_video(media.read_video(out_file))


## End-to-end meta-evolution: 10000 steps

In [ ]:
agent_model = "extended" #@param ['minimal', 'extended']
agent_logic = BasicAgentLogic(config, minimal_net=agent_model=="minimal")

mutator_type = "randomly_adaptive" #@param ['basic', 'randomly_adaptive']
sd = 1e-2 if mutator_type == "basic" and agent_model == "basic" else 1e-3
mutator = (BasicMutator(sd=sd, change_perc=0.2) if mutator_type == "basic"
           else RandomlyAdaptiveMutator(init_sd=sd, change_perc=0.2))

In [ ]:
def count_agents_f(env, etd):
  return etd.is_agent_fn(env.type_grid).sum()

@partial(jit, static_argnames=["config", "agent_logic", "mutator", "n_steps", "n_max_programs"])
def evaluate_biome(key, st_env, config, agent_logic, mutator, n_steps,
                   init_program=None, n_max_programs=128):
  def body_f(i, carry):
    key, env, programs, tot_agents_n = carry
    ku, key = jr.split(key)

    env, programs = step_env(
        ku, env, config, agent_logic, programs, do_reproduction=True,
          mutate_programs=True, mutator=mutator)

    tot_agents_n += count_agents_f(env, config.etd)
    return key, env, programs, tot_agents_n

  if init_program is None:
    ku, key = jr.split(key)
    programs = vmap(agent_logic.initialize)(jr.split(ku, n_max_programs))
    ku, key = jr.split(key)
    programs = vmap(mutator.initialize)(jr.split(ku, programs.shape[0]), programs)
  else:
    programs = jp.repeat(init_program[None, :], n_max_programs, axis=0)

  key, env, programs, tot_agents_n = jax.lax.fori_loop(
      0, n_steps, body_f, (key, st_env, programs, 0))

  # check whether they got extinct:
  is_extinct = (count_agents_f(env, config.etd) == 0).astype(jp.int32)
  return tot_agents_n, is_extinct

In [ ]:
from evojax.algo import PGPE

key = jr.PRNGKey(137)

N_MAX_PROGRAMS = 128

n_steps = 10000 # 2000

# initialize params
ku, key = jr.split(key)
init_program = agent_logic.initialize(ku)
ku, key = jr.split(key)
init_program = mutator.initialize(ku, init_program)


pop_size = 32
ku, key = jr.split(key)
solver = PGPE(
    pop_size=pop_size,
    param_size=init_program.shape[0],
    optimizer='adam',
    center_learning_rate=0.001,
    stdev_learning_rate=0.001,
    stdev_max_change=0.002,
    seed=ku[0],
    init_params=init_program,
    init_stdev=0.001,
)

n_max_programs = 64 # less than usual, but it doesn't really matter.
death_penalty = 1e6 * n_steps // 1000

@jit
def v_fitness_f(key, v_params):
  ku, key = jr.split(key)
  tot_agents_n, num_deaths = vmap(partial(
      evaluate_biome, st_env=st_env, config=config, agent_logic=agent_logic,
      mutator=mutator, n_steps=n_steps, n_max_programs=n_max_programs))(
          jr.split(key, pop_size), init_program=v_params)
  fitness = tot_agents_n - num_deaths * death_penalty
  return fitness

mean_fit_log = []
max_fit_log = []

In [ ]:

for _ in range(30):
  # sample
  programs = solver.ask()
  # eval
  key, k1 = jr.split(key)
  fitness = v_fitness_f(k1, programs)
  # update
  solver.tell(fitness)

  mean_fitness = fitness.mean()
  max_fitness = fitness.max()
  mean_fit_log.append(mean_fitness)
  max_fit_log.append(max_fitness)
  print(mean_fitness, max_fitness)
  if len(mean_fit_log) % 10 == 0:
    clear_output()
    plt.plot(mean_fit_log, label="mean_fitness")
    plt.plot(max_fit_log, label="max_fitness")
    plt.legend()
    plt.show()

### Show an example run of the result

Consider modifying the code to vary the extent of the simulation and video configs.

In [ ]:

key = jr.PRNGKey(43)

# How many unique programs (organisms) are allowed in the simulation.
N_MAX_PROGRAMS = 128

n_steps = 10000

programs = jp.repeat(solver.best_params[None,:], N_MAX_PROGRAMS, axis=0)

ku, key = jr.split(key)
visualize_run(ku, programs, agent_logic, mutator, st_env, config, n_steps)


## End-to-end meta-evolution: 15k, higher extinction penalty

In [ ]:
agent_model = "extended" #@param ['minimal', 'extended']
agent_logic = BasicAgentLogic(config, minimal_net=agent_model=="minimal")

mutator_type = "randomly_adaptive" #@param ['basic', 'randomly_adaptive']
sd = 1e-2 if mutator_type == "basic" and agent_model == "basic" else 1e-3
mutator = (BasicMutator(sd=sd, change_perc=0.2) if mutator_type == "basic"
           else RandomlyAdaptiveMutator(init_sd=sd, change_perc=0.2))

In [ ]:
def count_agents_f(env, etd):
  return etd.is_agent_fn(env.type_grid).sum()

@partial(jit, static_argnames=["config", "agent_logic", "mutator", "n_steps", "n_max_programs"])
def evaluate_biome(key, st_env, config, agent_logic, mutator, n_steps,
                   init_program=None, n_max_programs=128):
  def body_f(i, carry):
    key, env, programs, tot_agents_n = carry
    ku, key = jr.split(key)

    env, programs = step_env(
        ku, env, config, agent_logic, programs, do_reproduction=True,
          mutate_programs=True, mutator=mutator)

    tot_agents_n += count_agents_f(env, config.etd)
    return key, env, programs, tot_agents_n

  if init_program is None:
    ku, key = jr.split(key)
    programs = vmap(agent_logic.initialize)(jr.split(ku, n_max_programs))
    ku, key = jr.split(key)
    programs = vmap(mutator.initialize)(jr.split(ku, programs.shape[0]), programs)
  else:
    programs = jp.repeat(init_program[None, :], n_max_programs, axis=0)

  key, env, programs, tot_agents_n = jax.lax.fori_loop(
      0, n_steps, body_f, (key, st_env, programs, 0))

  # check whether they got extinct:
  is_extinct = (count_agents_f(env, config.etd) == 0).astype(jp.int32)
  return tot_agents_n, is_extinct

In [ ]:
from evojax.algo import PGPE

key = jr.PRNGKey(138)

N_MAX_PROGRAMS = 128

n_steps = 15000

# initialize params
ku, key = jr.split(key)
init_program = agent_logic.initialize(ku)
ku, key = jr.split(key)
init_program = mutator.initialize(ku, init_program)


pop_size = 32
ku, key = jr.split(key)
solver = PGPE(
    pop_size=pop_size,
    param_size=init_program.shape[0],
    optimizer='adam',
    center_learning_rate=0.001,
    stdev_learning_rate=0.001,
    stdev_max_change=0.002,
    seed=ku[0],
    init_params=init_program,
    init_stdev=0.001,
)

n_max_programs = 64 # less than usual, but it doesn't really matter.
death_penalty = 1e6 * n_steps // 1000

@jit
def v_fitness_f(key, v_params):
  ku, key = jr.split(key)
  tot_agents_n, num_deaths = vmap(partial(
      evaluate_biome, st_env=st_env, config=config, agent_logic=agent_logic,
      mutator=mutator, n_steps=n_steps, n_max_programs=n_max_programs))(
          jr.split(key, pop_size), init_program=v_params)
  fitness = tot_agents_n - num_deaths * death_penalty
  return fitness

mean_fit_log = []
max_fit_log = []

In [ ]:

for _ in range(30):
  # sample
  programs = solver.ask()
  # eval
  key, k1 = jr.split(key)
  fitness = v_fitness_f(k1, programs)
  # update
  solver.tell(fitness)

  mean_fitness = fitness.mean()
  max_fitness = fitness.max()
  mean_fit_log.append(mean_fitness)
  max_fit_log.append(max_fitness)
  print(mean_fitness, max_fitness)
  if len(mean_fit_log) % 10 == 0:
    clear_output()
    plt.plot(mean_fit_log, label="mean_fitness")
    plt.plot(max_fit_log, label="max_fitness")
    plt.legend()
    plt.show()

### Show an example run of the result

Consider modifying the code to vary the extent of the simulation and video configs.

In [ ]:

key = jr.PRNGKey(44)

# How many unique programs (organisms) are allowed in the simulation.
N_MAX_PROGRAMS = 128

n_steps = 15000

programs = jp.repeat(solver.best_params[None,:], N_MAX_PROGRAMS, axis=0)

ku, key = jr.split(key)
visualize_run(ku, programs, agent_logic, mutator, st_env, config, n_steps)


### Widen the environment: does anything change?

In [ ]:

test_env_and_config = evm.get_env_and_config(ec_id, width_type=w*4, h=h)
env, _ = test_env_and_config

key = jr.PRNGKey(45)

# How many unique programs (organisms) are allowed in the simulation.
N_MAX_PROGRAMS = 128

n_steps = 30000

zoom_sz = 24

programs = jp.repeat(solver.best_params[None,:], N_MAX_PROGRAMS, axis=0)

ku, key = jr.split(key)
visualize_run(ku, programs, agent_logic, mutator, env, config, n_steps,
              zoom_sz=zoom_sz)


## Petri dish meta-evolution: aiming at a target size


In [ ]:

def visualize_petri_run(key, programs, agent_logic, st_env, config, n_steps,
                  init_steps_per_frame=2,
                  when_to_double_speed=[100, 500, 1000, 2000, 5000],
                  fps=20,
                  zoom_sz=40):
  env = st_env
  steps_per_frame = init_steps_per_frame

  def make_frame(env):
    return zoom(evm.grab_image_from_env(env, config), zoom_sz)

  frame = make_frame(env)

  out_file = "video.mp4"
  with media.VideoWriter(out_file, shape=frame.shape[:2], fps=fps,
                         crf=18) as video:
    for i in tqdm.trange(n_steps):
      if i in when_to_double_speed:
        steps_per_frame *= 2

      key, ku = jr.split(key)
      env, n_successful_repr = step_env(
          ku, env, config, agent_logic, programs, do_reproduction=True,
          mutate_programs=False, intercept_reproduction=True,
          min_repr_energy_requirement=jp.zeros([2]) # we don't care about it here.
          )

      if i % steps_per_frame == 0:
        video.add_image(make_frame(env))

  media.show_video(media.read_video(out_file))

In [ ]:
ec_id = "persistence" #@param ['persistence', 'pestilence', 'collaboration', 'sideways']
env_width_type = "landscape" #@param ['wide', 'landscape', 'square', 'petri']
# original h is 72
h = 16 #@param {type: "integer"}

env_and_config = evm.get_env_and_config(ec_id, width_type=env_width_type, h=h)
st_env, config = env_and_config
w = st_env.type_grid.shape[1]

agent_model = "extended" #@param ['minimal', 'extended']
agent_logic = BasicAgentLogic(config, minimal_net=agent_model=="minimal")

mutator_type = "basic" #@param ['basic', 'randomly_adaptive']
sd = 1e-2 if mutator_type == "basic" and agent_model == "basic" else 1e-3
mutator = (BasicMutator(sd=sd, change_perc=0.2) if mutator_type == "basic"
           else RandomlyAdaptiveMutator(init_sd=sd, change_perc=0.2))

### Show an example petri run

In [ ]:
# visualize the result in a petri world first.
key = jr.PRNGKey(43)

# How many unique programs (organisms) are allowed in the simulation.
N_MAX_PROGRAMS = 128

n_steps = 2000

# initialize params
ku, key = jr.split(key)
init_program = agent_logic.initialize(ku)
programs = jp.repeat(init_program[None,:], N_MAX_PROGRAMS, axis=0)

ku, key = jr.split(key)
visualize_petri_run(ku, programs, agent_logic, st_env, config, n_steps)


### Meta evolve with petri

In [ ]:
from evojax.algo import PGPE

key = jr.PRNGKey(137)

N_MAX_PROGRAMS = 128

n_steps = 2000

# initialize params
ku, key = jr.split(key)
init_program = agent_logic.initialize(ku)
# make sure the mutator doesn't actually increase # of parameters here, because
# we are evolving without mutators.
# essentially, we could remove this line.
ku, key = jr.split(key)
init_program = mutator.initialize(ku, init_program)


pop_size = 64 # more, because anyway this is faster.
ku, key = jr.split(key)
solver = PGPE(
    pop_size=pop_size,
    param_size=init_program.shape[0],
    optimizer='adam',
    center_learning_rate=0.001,
    stdev_learning_rate=0.001,
    stdev_max_change=0.002,
    seed=ku[0],
    init_params=init_program,
    init_stdev=0.001,
)

n_max_programs = 1 # No reproduction occurs, after all.

# how many agents we want in the environment at any point in time.
n_agent_target = 50
min_repr_energy_requirement = (config.dissipation_per_step * 4) + config.specialize_cost * 2

def count_agents_f(env, etd):
  return etd.is_agent_fn(env.type_grid).sum()


@jit
def fitness_f(key, program):
  def body_f(i, carry):
    key, env, fit = carry
    key, keyused = jr.split(key)

    new_env, n_successful_repr = step_env(
        keyused, env, config, agent_logic, program[None,:], do_reproduction=True,
          mutate_programs=False, intercept_reproduction=True,
        min_repr_energy_requirement=min_repr_energy_requirement)

    n_agent_fit = -jp.abs(n_agent_target - count_agents_f(new_env, config.etd))
    repr_fit = n_successful_repr * n_agent_target * 4

    fit = fit + n_agent_fit + repr_fit
    return key, new_env, fit

  # process the integrity first
  env = process_structural_integrity_n_times(st_env, config, 10)
  _, f_env, fit = jax.lax.fori_loop(0, n_steps, body_f, (key, env, 0.))
  return fit / n_steps

@jit
def v_fitness_f(key, v_params):
  return vmap(fitness_f)(jr.split(key, pop_size), v_params)

mean_fit_log = []
max_fit_log = []

In [ ]:

for _ in range(30):
  # sample
  programs = solver.ask()
  # eval
  key, k1 = jr.split(key)
  fitness = v_fitness_f(k1, programs)
  # update
  solver.tell(fitness)

  mean_fitness = fitness.mean()
  max_fitness = fitness.max()
  mean_fit_log.append(mean_fitness)
  max_fit_log.append(max_fitness)
  print(mean_fitness, max_fitness)
  if len(mean_fit_log) % 10 == 0:
    clear_output()
    plt.plot(mean_fit_log, label="mean_fitness")
    plt.plot(max_fit_log, label="max_fitness")
    plt.legend()
    plt.show()

### meta-evolved petri run

In [ ]:
# visualize the result in a petri world first.
key = jr.PRNGKey(43)

# How many unique programs (organisms) are allowed in the simulation.
N_MAX_PROGRAMS = 128

n_steps = 2000

programs = jp.repeat(solver.best_params[None,:], N_MAX_PROGRAMS, axis=0)

ku, key = jr.split(key)
visualize_petri_run(ku, programs, agent_logic, st_env, config, n_steps)


### test it without mutations, for long.

In [ ]:

key = jr.PRNGKey(43)

# How many unique programs (organisms) are allowed in the simulation.
N_MAX_PROGRAMS = 128

n_steps = 20000

programs = jp.repeat(solver.best_params[None,:], N_MAX_PROGRAMS, axis=0)


env = st_env

steps_per_frame = 2
# on what STEP to double speed.
# NOTE that it is different from previous colabs, where we were talking about
# frames.
when_to_double_speed = [100, 500, 1000, 2000, 5000]
fps = 20
# this affects the size of the image. If this number is not even, the resulting
# video *may* not be supported by all renderers.
zoom_sz = 40

def make_frame(env):
  return zoom(evm.grab_image_from_env(env, config), zoom_sz)

frame = make_frame(env)

out_file = "video.mp4"
with media.VideoWriter(out_file, shape=frame.shape[:2], fps=fps,
                       crf=18,
                       ) as video:
  for i in tqdm.trange(n_steps):
    if i in when_to_double_speed:
      steps_per_frame *= 2

    key, ku = jr.split(key)

    env = step_env(
        ku, env, config, agent_logic, programs, do_reproduction=True,
          mutate_programs=False, mutator=mutator)

    if i % steps_per_frame == 0:
      video.add_image(make_frame(env))

media.show_video(media.read_video(out_file))


### test it with mutations, for long.

In [ ]:

key = jr.PRNGKey(43)

# How many unique programs (organisms) are allowed in the simulation.
N_MAX_PROGRAMS = 128

n_steps = 20000

programs = jp.repeat(solver.best_params[None,:], N_MAX_PROGRAMS, axis=0)


mutator_type = "basic"
sd = 1e-2 if mutator_type == "basic" and agent_model == "basic" else 1e-3
mutator = (BasicMutator(sd=sd, change_perc=0.2) if mutator_type == "basic"
           else RandomlyAdaptiveMutator(init_sd=sd, change_perc=0.2))

ku, key = jr.split(key)
visualize_run(ku, programs, agent_logic, mutator, st_env, config, n_steps)


### test it with small mutations, for long.

In [ ]:

key = jr.PRNGKey(43)

# How many unique programs (organisms) are allowed in the simulation.
N_MAX_PROGRAMS = 128

n_steps = 20000

programs = jp.repeat(solver.best_params[None,:], N_MAX_PROGRAMS, axis=0)

mutator_type = "basic"
sd = 1e-4
mutator = (BasicMutator(sd=sd, change_perc=0.2) if mutator_type == "basic"
           else RandomlyAdaptiveMutator(init_sd=sd, change_perc=0.2))

ku, key = jr.split(key)
visualize_run(ku, programs, agent_logic, mutator, st_env, config, n_steps)


### Widen the environment: does anything change?

In [ ]:

test_env_and_config = evm.get_env_and_config(ec_id, width_type=w*4, h=h)
env, _ = test_env_and_config

key = jr.PRNGKey(45)

# How many unique programs (organisms) are allowed in the simulation.
N_MAX_PROGRAMS = 128

n_steps = 50000

# on what FRAME to double speed.
when_to_double_speed = [100, 500, 1000, 2000, 5000, 15000]

# get the program from the solver.
programs = jp.repeat(solver.best_params[None,:], N_MAX_PROGRAMS, axis=0)

ku, key = jr.split(key)
visualize_run(ku, programs, agent_logic, mutator, env, config, n_steps,
              when_to_double_speed=when_to_double_speed,
              zoom_sz=zoom_sz)

### Save the dna for future use

In [ ]:
notes = "This dna was trained with petri meta evolution. So, no mutator was used. But I tested and it definitely works with BasicMutators and sd of 1e-4. In some cases, 1e-3. See the notebook evolving_in_small_worlds.ipynb for more context." # @param {type:"string"}
author = "anonymous" # @param {type:"string"}

out_f = save_dna(
    dna, ec_id, config, agent_logic, mutator, env_h=h, env_w=w,
    author=author, notes=notes, out_dir="./")

In [ ]:
from google.colab import files
files.download(out_f + ".npy")
files.download(out_f + ".txt")

## Extra: load a saved dna and run it

In [ ]:
load_from_out_f = True # @param ["True", "False"] {type:"raw"}
file_path_if_false = "" # @param {type:"string"}

# if you want to load a prepackaged dna, set load_from_this_package to True manually.
dna = load_dna(out_f if load_from_out_f else file_path_if_false,
               load_from_this_package=False)

In [ ]:
test_env_and_config = evm.get_env_and_config(ec_id, width_type=w*4, h=h)
env, _ = test_env_and_config

key = jr.PRNGKey(45)

# How many unique programs (organisms) are allowed in the simulation.
N_MAX_PROGRAMS = 128

n_steps = 50000

# on what FRAME to double speed.
when_to_double_speed = [100, 500, 1000, 2000, 5000, 15000]

# get the program from the solver.
programs = jp.repeat(dna[None,:], N_MAX_PROGRAMS, axis=0)
zoom_sz = 40

ku, key = jr.split(key)
visualize_run(ku, programs, agent_logic, mutator, env, config, n_steps,
              when_to_double_speed=when_to_double_speed,
              zoom_sz=zoom_sz)